In [58]:
import pandas as pd
import janitor

In [59]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [60]:
data = pd.read_csv('../latest_raw.csv', encoding='utf-8')
data = data.drop(['MUNICIPIO_RES'], axis=1) # readily drop this column

In [61]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [62]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [63]:
len(data)

292124

### Note
Daily report for april 20 (ie, reported in 21 april) stops using testing site (ENTIDAD_UM),
<br>
And starts using region of residence (ENTIDAD_RES) so we shall treat both sets of data accordingly

In [64]:
dat = data.copy()

In [65]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [66]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-21','2022-12-31') # the long con
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [67]:
tbl = pd.concat([tbl_x, tbl_y])

In [68]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,15
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,13
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,7
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-01-28,2020-01-25,9999-99-99,3,146a80,Local,1
...,...,...,...,...,...,...,...
11842,2020-05-03,2020-05-01,9999-99-99,25,192d9b,Local,1
11843,2020-05-03,2020-05-02,2020-05-03,27,1651ee,Local,1
11844,2020-05-03,2020-05-02,9999-99-99,14,0db6a0,Local,1
11845,2020-05-03,2020-05-02,9999-99-99,31,04bc6f,Local,1


In [69]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [70]:
tbl['Status'] = '1'

In [71]:
tbl

,Date_Confirmed,Date_Symptoms,FECHA_DEF,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,9999-99-99,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
4,2020-01-28,2020-01-25,9999-99-99,3,146a80,Local,1
...,...,...,...,...,...,...,...
11842,2020-05-03,2020-05-01,9999-99-99,25,192d9b,Local,1
11843,2020-05-03,2020-05-02,2020-05-03,27,1651ee,Local,1
11844,2020-05-03,2020-05-02,9999-99-99,14,0db6a0,Local,1
11845,2020-05-03,2020-05-02,9999-99-99,31,04bc6f,Local,1


In [72]:
#tbl[~tbl.FECHA_DEF.str.startswith(('9999'))]
tbl = tbl[tbl.FECHA_DEF.str.startswith(('9999'))] # Not deceased

In [73]:
tbl = tbl.drop(['FECHA_DEF'], axis=1)

In [74]:
tbl

,Date_Confirmed,Date_Symptoms,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,30,1c29f8,Local,1
2,2020-01-14,2020-01-14,1,0ceb0f,Local,1
3,2020-01-27,2020-01-27,15,1879e0,Local,1
4,2020-01-28,2020-01-25,3,146a80,Local,1
...,...,...,...,...,...,...
11841,2020-05-03,2020-05-01,25,0d5f1e,Local,1
11842,2020-05-03,2020-05-01,25,192d9b,Local,1
11844,2020-05-03,2020-05-02,14,0db6a0,Local,1
11845,2020-05-03,2020-05-02,31,04bc6f,Local,1


In [75]:
geo = pd.read_csv('../data/geo/entidades.csv')

In [76]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [77]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [78]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [79]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Region', 'Region_ID', 'Region_Key', 'Origin']]

In [80]:
output

,Date_Confirmed,Date_Symptoms,Region,Region_ID,Region_Key,Origin
0,2020-01-06,2020-01-06,NAYARIT,18,NT,Local
1,2020-03-18,2020-03-17,NAYARIT,18,NT,Local
2,2020-03-19,2020-03-10,NAYARIT,18,NT,Local
3,2020-03-23,2020-03-19,NAYARIT,18,NT,Local
4,2020-03-24,2020-03-21,NAYARIT,18,NT,Local
...,...,...,...,...,...,...
24339,2020-05-02,2020-04-18,TLAXCALA,29,TL,Local
24340,2020-05-02,2020-04-24,TLAXCALA,29,TL,Local
24341,2020-05-02,2020-04-27,TLAXCALA,29,TL,Local
24342,2020-05-02,2020-04-29,TLAXCALA,29,TL,Local


In [81]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACÁN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [82]:
output = output.sort_values(['Date_Confirmed', 'Date_Symptoms', 'Region'])

In [83]:
output.Region.unique()

array(['NAYARIT', 'VERACRUZ', 'AGUASCALIENTES', 'MÉXICO',
       'BAJA CALIFORNIA SUR', 'GUANAJUATO', 'BAJA CALIFORNIA',
       'CIUDAD DE MÉXICO', 'COAHUILA', 'SINALOA', 'CHIAPAS', 'YUCATÁN',
       'MORELOS', 'NUEVO LEÓN', 'QUERÉTARO', 'PUEBLA', 'DURANGO',
       'JALISCO', 'QUINTANA ROO', 'CAMPECHE', 'SAN LUIS POTOSÍ',
       'GUERRERO', 'OAXACA', 'CHIHUAHUA', 'COLIMA', 'SONORA',
       'TAMAULIPAS', 'HIDALGO', 'TABASCO', 'MICHOACAN', 'ZACATECAS',
       'TLAXCALA'], dtype=object)

In [84]:
output.Origin.unique()

array(['Local', 'Alemania', 'Estados Unidos de América',
       'Estados Unidos de Am�rica', 'República de Honduras',
       'Rep�blica de Honduras', 'Venezuela', 'Cuba', 'Guatemala', 'Otro',
       'Perú', 'Per�', 'Camerún', 'Camer�n', 'Chile', 'El Salvador',
       'Canad�', 'Brasil', 'Colombia', 'Ecuador', 'Hait�'], dtype=object)

In [85]:
output.to_csv('../latest.csv', index=False)